<a href="https://colab.research.google.com/github/SHADES2003/SHADES2003/blob/main/animal_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [71]:
import os
import tensorflow as tf
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.layers import Dense, Conv2D, GlobalAveragePooling2D,Input # Changed conv2D to Conv2D
from tensorflow.keras.preprocessing.image import load_img,ImageDataGenerator
from tensorflow.keras import callbacks,optimizers
import numpy as np
from google.colab import drive

In [3]:
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
%cd drive/MyDrive/animal_classification/

/content/drive/MyDrive/animal_classification


In [ ]:
# !unzip archive\ \(6\).zip

In [9]:
from translate import translate

In [10]:
translate

{'cane': 'dog',
 'cavallo': 'horse',
 'elefante': 'elephant',
 'farfalla': 'butterfly',
 'gallina': 'chicken',
 'gatto': 'cat',
 'mucca': 'cow',
 'pecora': 'sheep',
 'scoiattolo': 'squirrel',
 'dog': 'cane',
 'elephant': 'elefante',
 'butterfly': 'farfalla',
 'chicken': 'gallina',
 'cat': 'gatto',
 'cow': 'mucca',
 'spider': 'ragno',
 'squirrel': 'scoiattolo'}

In [11]:
for i in os.listdir("raw-img"):
  try:
      os.rename("raw-img/"+i,"raw-img/"+translate[i])
  except Exception as e:
    print(e)

os.rename("raw-img/ragno","raw-img/spider")
# os.rename("raw-img/scoiattolo","raw-img/squirrel")

'ragno'


In [12]:
!ls raw-img/

butterfly  cat	chicken  cow  dog  elephant  horse  sheep  spider  squirrel


In [13]:
for i in os.listdir("raw-img"):
  print(i,len(os.listdir("raw-img/"+i)))

dog 4863
horse 2623
elephant 1446
butterfly 2112
chicken 3098
cat 1668
cow 1866
sheep 1820
spider 4821
squirrel 1862


In [16]:
try:
  os.mkdir("train")
  os.mkdir("test")
except:
  pass
for i in os.listdir("raw-img"):
  try:
    os.mkdir("train/"+i)
    os.mkdir("test/"+i)
  except:
    pass
  for j in os.listdir("raw-img/"+i)[:1000]:
    os.rename("raw-img/"+i+"/"+j,"train/"+i+"/"+j)
  for j in os.listdir("raw-img/"+i)[:400]:
    os.rename("raw-img/"+i+"/"+j,"test/"+i+"/"+j)


In [34]:
def img_Data(dir_path, target_size, batch, preprocessing):
  if preprocessing:
    gen_object = ImageDataGenerator(preprocessing_function=preprocessing)
  else:
    gen_object = ImageDataGenerator()
  return(gen_object.flow_from_directory(dir_path,
                                        target_size=target_size,
                                        batch_size=batch,
                                        class_mode="sparse",
                                        # class_list=class_lst,  # Remove this line as it's causing the error
                                        shuffle=True))

In [35]:
train_data_gen = img_Data("train",(224,224),500, preprocess_input)
valid_data_gen= img_Data("test",(224,224),500, preprocess_input)

Found 12446 images belonging to 10 classes.
Found 6286 images belonging to 10 classes.


In [81]:
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(224,224,3),alpha=1.0,include_top=False,weights='imagenet',
    input_tensor=None,pooling=None,
    classes=1000,
    classifier_activation='softmax')

In [82]:
base_model.trainable =  False

In [88]:
model = tf.keras.models.Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(1024, activation="relu"))
model.add(Dense(10, activation="softmax"))
model.compile(optimizer="adam",loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [89]:
elst =callbacks.EarlyStopping(monitor="val_loss",patience=5,mode="min")
save_ek =callbacks.ModelCheckpoint('.mdl_wt.keras',save_best_only=True,monitor="val_loss",mode="min")
# Change the file extension to '.keras' to save the entire model

In [90]:
model.fit(train_data_gen,batch_size=500,validation_data=valid_data_gen,callbacks=[elst,save_ek],epochs=10)

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


25/25 ━━━━━━━━━━━━━━━━━━━━ 1008s 39s/step - accuracy: 0.7205 - loss: 0.8540 - val_accuracy: 0.9443 - val_loss: 0.1909
Epoch 2/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 997s 39s/step - accuracy: 0.9589 - loss: 0.1360 - val_accuracy: 0.9535 - val_loss: 0.1545
Epoch 3/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 997s 39s/step - accuracy: 0.9776 - loss: 0.0834 - val_accuracy: 0.9586 - val_loss: 0.1423
Epoch 4/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 999s 39s/step - accuracy: 0.9843 - loss: 0.0618 - val_accuracy: 0.9531 - val_loss: 0.1679
Epoch 5/10
 1/25 ━━━━━━━━━━━━━━━━━━━━ 20:29 51s/step - accuracy: 0.9920 - loss: 0.0470

KeyboardInterrupt: 

# New Section